## Target: find a local quantum channel that inverses dthe local ephasing channel

First we load in the MPS calculation results from ITensors calculation since this is the only package that I know would calculate PBC DMRG

In [1]:
# Jul. 1
#@YZhang
import numpy as np
import math
#from scipy import linalg as la
from numpy import linalg as la
import random
import itertools
import time
import h5py
import quimb.tensor as qtn
import quimb as qu
import torch
from vqe_functions import*
#Implement a vqe circuit


from torch import optim
import tqdm
import cotengra as ctg

from quimb.tensor.decomp import _ISOMETRIZE_METHODS
from autoray import (
    backend_like,
    compose,
    do,
    reshape,
)
opti = ctg.ReusableHyperOptimizer(
    progbar=True,
    methods=['greedy'],
    reconf_opts={},
    max_repeats=32, 
    optlib='random',
    # directory=  # set this for persistent cache
)
#opti = None
@compose
def isometrize_qr_fixed(x, backend=None):
    """Perform isometrization using the QR decomposition. FIX FOR NEW PYTORCH"""
    with backend_like(backend):
        Q, R = do("linalg.qr", x)
        # stabilize qr by fixing diagonal of R in canonical, positive form (we
        # don't actaully do anything to R, just absorb the necessary sign -> Q)
        rd = do("diag", R)
        s = do("sgn", rd) + (rd == 0)
        Q = Q * reshape(s, (1, -1))
        return Q
_ISOMETRIZE_METHODS["qr"] = isometrize_qr_fixed

# Function to load MPS tensors from HDF5
def load_mps_from_hdf5(filename):
    mps_tensors = []
    with h5py.File(filename, "r") as f:
        for i in range(len(f.keys())):
            mps_tensors.append(np.array(f[f'tensor_{i+1}']))
    return mps_tensors

def construct_mps(mps_tensors):
    tn = qtn.TensorNetwork()
    for i in range(0,len(mps_tensors)):
        # wtf is the indexing in ITensors? so weird. I have to make several cases
        if i ==0:
            inds = ('p0', 'b0',)# p for 'physical' and b for 'bond' 
        elif i ==len(mps_tensors)-1:
            inds = (f'b{i-1}',f'p{i}')
        elif i ==1:
            inds = ('b0','p1', 'b1')
        else:
            inds = (f'b{i-1}',f'b{i}',f'p{i}')
        tensor = qtn.Tensor(mps_tensors[i], inds,tags='MPS')
        tn = tn&tensor
    return tn

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
# test and check nomalization; grateful that it worked out!
L = 8
mps_tensors = load_mps_from_hdf5(f"results/mps_data{L}.h5")
gs = construct_mps(mps_tensors)

In [3]:
p0 = np.array([[ 1.,  0.,],[ 0,  1]])
p1 = np.array([[ 0.,  1.,],[ 1,  0]])
p2 = np.array([[ 0, -1j],[1j, 0]],dtype = 'complex128')
p3 = np.array([[ 1.,  0.,],[ 0, -1]])
paulis = [p0,p1,p2,p3]

#tn =qtn.MPS_rand_state(L=L, bond_dim=2, dtype = 'complex128')
def apply_channel(mps,pauli =p0,p_channel =0):
    #let's now make some noise!
    mps_c = mps.H
    mps_c = mps_c.reindex({f'p{i}':f'pc{i}' for i in range(len(mps.tensors))})
    rho= mps&mps_c#)^all
    
    p_rel_tensor = np.zeros([2,2,2])+0j
    p_rel_tensor[0] = np.eye(2)
    p_rel_tensor[1] = pauli
#
    pc_rel_tensor = np.zeros([2,2,2])+0j
    pc_rel_tensor[0] = (1-p_channel)*np.eye(2)
    pc_rel_tensor[1] = p_channel*pauli
    #noise = p_channel*np.kron(pauli,pauli).reshape(2,2,2,2)
    
    #delta = np.einsum('ij,jl,jn->iln', np.eye(2,2), np.eye(2,2),np.eye(2,2))

    for i in range (len(mps.tensors)):
        #ind = f'k{i}',f'kc{i}',f'kc_ch{i}',f'k_ch{i}',
        #t = qtn.Tensor(noise,ind,tags=f'site{i}')
        inds_d = f'a{i}',f'p{i}',f'p_ch{i}',#ch for 'channelled'
        t = qtn.Tensor(p_rel_tensor,inds_d,tags='N')
        inds_dc = f'a{i}',f'pc_ch{i}',f'pc{i}',
        tc = qtn.Tensor(pc_rel_tensor,inds_dc,tags='N')
        #d1_ind =f'a{i}',f'ac{i}',f'l{i}',
        #d1 = qtn.Tensor(delta,d1_ind,tags='del')
        #prob = qtn.Tensor([1-p_channel,p_channel], inds = (f'l{i}',))
        rho = rho&t&tc#&d1&prob

    return rho

rho = apply_channel(gs,pauli =p2,p_channel =.11)
rho@rho

0.179555247932146

### now we need to define a channel-inversion circuit. we use a local quantum circuit for this

In [4]:
# let's use the order: physical, ancilla, ....
def overlap(pqc: qtn.TensorNetwork,rho,gs):
    for i in range (num_qubits):
        #if i<num_qubits//2:
        #    pqc = pqc.reindex({f'k{i}':f'p_ch{i}'})
        #    pqc = pqc.reindex({psi_pqc.tensors[i].inds[-1]:f'p_gs{i}'})
        #else:
        #    pqc = pqc.reindex({f'k{i}':f'a_in{i-num_qubits//2}'})
        #    pqc = pqc.reindex({psi_pqc.tensors[i].inds[-1]:f'a_out{i-num_qubits//2}'})
        if i%2:
            pqc = pqc.reindex({f'k{i}':f'p_ch{i//2}'})
            pqc = pqc.reindex({psi_pqc.tensors[i].inds[-1]:f'p_gs{i//2}'})
        else:
            pqc = pqc.reindex({f'k{i}':f'a_in{i//2}'})
            pqc = pqc.reindex({psi_pqc.tensors[i].inds[-1]:f'a_out{i//2}'})
    
    pqc_c = pqc.H
    for i in range (num_qubits//2):
        pqc_c = pqc_c.reindex({f'a_in{i}':f'ac_in{i}'})
        pqc_c = pqc_c.reindex({f'p_gs{i}':f'pc_gs{i}'})
        pqc_c = pqc_c.reindex({f'p_ch{i}':f'pc_ch{i}'})
        
    for i in range(L):
        prod = qtn.Tensor(np.array([1,0]),inds = (f'a_in{i}',),tags = 'A')
        prod_c = qtn.Tensor(np.array([1,0]),inds = (f'ac_in{i}',),tags = 'A')
        prod.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
        prod_c.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
        rho = rho&prod&prod_c
    gs_c = gs.H
    gs = gs.reindex({f'p{i}':f'p_gs{i}' for i in range(L)})
    gs_c = gs_c.reindex({f'p{i}':f'pc_gs{i}' for i in range(L)})
    #(gs_c&gs&rho&pqc_c&pqc).draw(['U','N','MPS','A'])
    return -1*abs((gs_c&gs&rho&pqc_c&pqc).contract(optimize=opti))

In [5]:
class TNModel(torch.nn.Module):

    def __init__(self, pqc,rho,gs):
        super().__init__()

        # extract the raw arrays and a skeleton of the TN
        params, self.skeleton = qtn.pack(pqc)
        # n.b. you might want to do extra processing here to e.g. store each
        # parameter as a reshaped matrix (from left_inds -> right_inds), for
        # some optimizers, and for some torch parametrizations
        self.torch_params = torch.nn.ParameterDict({
            # torch requires strings as keys
            str(i): torch.nn.Parameter(initial)
            for i, initial in params.items()
        })
        self._loss_fn = lambda x: overlap(x,rho,gs)

    def forward(self):
        # convert back to original int key format
        params = {int(i): p for i, p in self.torch_params.items()}
        # reconstruct the TN with the new parameters
        pqc = qtn.unpack(params, self.skeleton)
        # isometrize and then return the energy
        return self._loss_fn(pqc.isometrize(method='qr'))

In [6]:
num_qubits = 2*L
depth = 2

rho.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
gs.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))

psi_pqc = qmps_f(num_qubits, in_depth= depth, n_Qbit=num_qubits-1, qmps_structure="brickwall", canon="left",val_iden = 0.0,rand = False)
pqc = psi_pqc.tensors[num_qubits]
for i in range (num_qubits+1,len(psi_pqc.tensors)):
    pqc = pqc&psi_pqc.tensors[i] #extrating the pqc part
pqc.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))

In [7]:
overlap(pqc,rho,gs)

  0%|                                                                                                                     | 0/32 [00:00<?, ?it/s]/opt/anaconda3/envs/py311/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/opt/anaconda3/envs/py311/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/opt/anaconda3/envs/py311/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/opt/anaconda3/envs/py311/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper opt

tensor(-0.3963, dtype=torch.float64)

In [8]:
psi_pqc = qmps_f(num_qubits, in_depth= depth, n_Qbit=num_qubits-1, qmps_structure="brickwall", canon="left",val_iden = 0.1,rand = True)
pqc = psi_pqc.tensors[num_qubits]
for i in range (num_qubits+1,len(psi_pqc.tensors)):
    pqc = pqc&psi_pqc.tensors[i] #extrating the pqc part
pqc.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
model = TNModel(pqc,rho,gs)
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=200, gamma=0.5)
num_steps = 500
pbar = tqdm.tqdm(range(num_steps))
previous_loss = torch.inf
losses = []
for step in pbar:
    optimizer.zero_grad()
    loss = model.forward()
    losses.append(loss.detach().numpy())
    loss.backward()
    optimizer.step()
    pbar.set_description(f"Loss={loss} - LR={lr}")
    if step > 100 and torch.abs(previous_loss - loss) < 1e-10:
        print("Early stopping loss difference is smaller than 1e-10")
        break
    previous_loss = loss.clone()
print(f'traning loss: {loss}')

Loss=-0.5490074602509422 - LR=0.01:  76%|█████████████████████████████████████████████████████▍                | 382/500 [00:25<00:07, 14.96it/s]

Early stopping loss difference is smaller than 1e-10
traning loss: -0.5490074602509422
